先在TensorFlow上实现多项逻辑斯谛回归练练手，然后增加难度实现基于神经网络的转移依存句法分析，试验Xavier初始化、Dropout和Adam优化器。最后推导RNN和语言模型的困惑度、梯度、反向传播和复杂度。


正式开始利用TensorFlow了，这里采用最新的TensorFlow-r1.2和Python2.7。如果在安装或升级TF的过程中遇到权限问题，比如：
`IOError: [Errno 13] Permission denied: '/usr/local/bin/markdown_py'`
不如试试：

`pip install tensorflow --user`

或[《从源码编译安装TensorFlow》](http://www.hankcs.com/ml/compile-and-install-tensorflow-from-source.html)，自行编译安装得到的库效率更高。

一些TensorFlow函数假定输入为行向量，所以乘上权值矩阵的时候必须右乘$xW+b$
）而不是左乘$(Wx+b)$。

In [3]:
import numpy as np
import tensorflow as tf
from utils.general_utils import test_all_close

# **1 Tensorflow Softmax**

以交叉熵损失函数:

$$
\begin{align} 
	J(\boldsymbol{W}) &= CE(\boldsymbol{y}, softmax(\boldsymbol{xW})) 
\end{align}
$$

实现线性分类器。要求使用TensorFlow的自动微分功能将模型拟合到数据。

## **1-1 softmax**

手写实现softmax，不要用直接tf.nn.softmax。如果输入是矩阵，则视作不相干的行向量集合。
softmax的公式如下：

$$
softmax{(x)_i}=\frac{e^{x_i}}{\sum_je^{x_j}}
$$

In [7]:
def softmax(x):
    """
    Compute the softmax function in tensorflow.
    You might find the tensorflow functions tf.exp, tf.reduce_max,
    tf.reduce_sum, tf.expand_dims useful. (Many solutions are possible, so you may
    not need to use all of these functions). Recall also that many common
    tensorflow operations are sugared (e.g. x * y does a tensor multiplication
    if x and y are both tensors). Make sure to implement the numerical stability
    fixes as in the previous homework!
    Args:
        x:   tf.Tensor with shape (n_samples, n_features). Note feature vectors are
                  represented by row-vectors. (For simplicity, no need to handle 1-d
                  input as in the previous homework)
    Returns:
        out: tf.Tensor with shape (n_sample, n_features). You need to construct this
                  tensor in this problem.
    """
    ### YOUR CODE HERE
    x_max = tf.reduce_max(x,1,keep_dims=True)          # find row-wise maximums
    x_sub = tf.subtract(x,x_max)                            # subtract maximums
    x_exp = tf.exp(x_sub)                              # exponentiation
    sum_exp = tf.reduce_sum(x_exp,1,keep_dims=True)    # row-wise sums
    out = tf.div(x_exp,sum_exp)                        # divide
    ### END YOUR CODE
    return out

In [8]:
#测试结果：

def test_softmax_basic():
    """
    Some simple tests of softmax to get you started.
    Warning: these are not exhaustive.
    """

    test1 = softmax(tf.constant(np.array([[1001, 1002], [3, 4]]), dtype=tf.float32))
    with tf.Session() as sess:
            test1 = sess.run(test1)
    test_all_close("Softmax test 1", test1, np.array([[0.26894142,  0.73105858],
                                                      [0.26894142,  0.73105858]]))

    test2 = softmax(tf.constant(np.array([[-1001, -1002]]), dtype=tf.float32))
    with tf.Session() as sess:
            test2 = sess.run(test2)
    test_all_close("Softmax test 2", test2, np.array([[0.73105858, 0.26894142]]))

    print("Basic (non-exhaustive) softmax tests pass\n")

test_softmax_basic()

Softmax test 1 passed!
Softmax test 2 passed!
Basic (non-exhaustive) softmax tests pass



## 1-2 交叉熵

在脚本`q1_softmax.py`中，通过TensorFlow来构造交叉熵损失函数（Cross Entropy Loss）。交叉熵损失函数的公式:

$$
CE(y,\hat{y})=-\sum_{i=1}^{N_c}{y_ilog(\hat{y}_i)}
$$

在这里$y\in\mathbb{R}^5$是一个one-hot标签向量，$N_c$是所有类目的数量。请注意，你不能使用`Tensorflow`内建的`cross-entropy`函数或者其他相关内建函数。

In [13]:
def cross_entropy_loss(y, yhat):
    """
    Compute the cross entropy loss in tensorflow.
    The loss should be summed over the current minibatch.
    y is a one-hot tensor of shape (n_samples, n_classes) and yhat is a tensor
    of shape (n_samples, n_classes). y should be of dtype tf.int32, and yhat should
    be of dtype tf.float32.
    The functions tf.to_float, tf.reduce_sum, and tf.log might prove useful. (Many
    solutions are possible, so you may not need to use all of these functions).
    Note: You are NOT allowed to use the tensorflow built-in cross-entropy
                functions.
    Args:
        y:    tf.Tensor with shape (n_samples, n_classes). One-hot encoded.
        yhat: tf.Tensorwith shape (n_sample, n_classes). Each row encodes a
                    probability distribution and should sum to 1.
    Returns:
        out:  tf.Tensor with shape (1,) (Scalar output). You need to construct this
                    tensor in the problem.
    """
    ### YOUR CODE HERE
    l_yhat = tf.log(yhat)                           # log yhat
    product = tf.multiply(tf.to_float(y), l_yhat)        # multiply element-wise
    out = tf.negative(tf.reduce_sum(product))            # negative summation to scalar
    ### END YOUR CODE
    return out

In [14]:
#测试结果：
#注意不同版本的TensorFlow在函数调用上可能有区别
def test_cross_entropy_loss_basic():
    """
    Some simple tests of cross_entropy_loss to get you started.
    Warning: these are not exhaustive.
    """
    y = np.array([[0, 1], [1, 0], [1, 0]])
    yhat = np.array([[.5, .5], [.5, .5], [.5, .5]])

    test1 = cross_entropy_loss(
            tf.constant(y, dtype=tf.int32),
            tf.constant(yhat, dtype=tf.float32))
    with tf.Session() as sess:
        test1 = sess.run(test1)
    expected = -3 * np.log(.5)
    test_all_close("Cross-entropy test 1", test1, expected)

    print("Basic (non-exhaustive) cross-entropy tests pass")
    
test_cross_entropy_loss_basic()

Cross-entropy test 1 passed!
Basic (non-exhaustive) cross-entropy tests pass


## **1-3  Placeholders & Feed Dictionaries**

仔细学习 `assignment2/model.py`脚本中的model类。并简要解释一下其中占位符变量 (place holder vaiables)和填充字典（feed dictionaries）函数的目的. 在`q1_classifier.py`中填充好`add_palceholders和creat_feed_dict`这两个函数。

提示: 请注意**配置变量**（configuration variables）在`config`类中已经储存好了，而在代码中你会用到它们。 

答案：

在Tensorflow 计算图中，占位符变量是作为其输入节点而存在的。而填充字典函数说明了在计算的时候怎么给这些占位符（或者其他）变量赋值。

## **1-4 Softmax & CE Loss**

在脚本`ql_classifier.py`中的完成一个用`softmax`进行分类的基本流程。并在同一个脚本中的函数`add_loss_op`中补充好交叉熵损失函数。 请注意，使用本题之前提供的函数，而不是Tensorflow 内建函数。


In [15]:
def add_prediction_op(self):
    """Adds the core transformation for this model which transforms a batch of input
    data into a batch of predictions. In this case, the transformation is a linear layer plus a
    softmax transformation:
    y = softmax(Wx + b)
    Hint: Make sure to create tf.Variables as needed.
    Hint: For this simple use-case, it's sufficient to initialize both weights W
                and biases b with zeros.
    Args:
        input_data: A tensor of shape (batch_size, n_features).
    Returns:
        pred: A tensor of shape (batch_size, n_classes)
    """
    ### YOUR CODE HERE
    with tf.variable_scope("softmax_transformation"):
        bias = tf.Variable(tf.random_uniform([self.config.n_classes]))
        W = tf.Variable(tf.random_uniform([self.config.n_features, self.config.n_classes]))
        z = tf.matmul(self.input_placeholder, W) + bias
    pred = softmax(z)
    ### END YOUR CODE
    return pred

#交叉熵损失函数
def add_loss_op(self, pred):
    """Adds cross_entropy_loss ops to the computational graph.
    Hint: Use the cross_entropy_loss function we defined. This should be a very
                short function.
    Args:
        pred: A tensor of shape (batch_size, n_classes)
    Returns:
        loss: A 0-d tensor (scalar)
    """
    ### YOUR CODE HERE
    loss = cross_entropy_loss(self.labels_placeholder, pred)
    ### END YOUR CODE
    return loss

## 1-5 Training Optimizer

在脚本`ql_classifier.py`中填充完成`add_training_op `函数。 并解释为什么Tensorflow 的自动微分功能能省去我们直接求梯度的工作。你可以在我们提供的数据上顺利运行`python ql_classifier.py`命令，并且确保能够跑通测试用例。这样你才能保证你的模型是合适的。 

提示：请一定要使用在config类中定义的学习率. 

答案：

只要正确定义好了计算图，Tensorflow就能自动应用反向传播算法来计算梯度。

In [16]:
def add_training_op(self, loss):
    """Sets up the training Ops.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train. See
    https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#Optimizer
    for more information.
    Hint: Use tf.train.GradientDescentOptimizer to get an optimizer object.
                Calling optimizer.minimize() will return a train_op object.
    Args:
        loss: Loss tensor, from cross_entropy_loss.
    Returns:
        train_op: The Op for training.
    """
    ### YOUR CODE HERE
    train_op = tf.train.GradientDescentOptimizer(self.config.lr).minimize(loss)
    ### END YOUR CODE
    return train_op

TF会自动求偏导数，GradientDescentOptimizer负责更新参数。

完整的代码运行见`ql_classifier.py`

运行后输出：
```
Epoch 0: loss = 111.91 (0.022 sec)
Epoch 1: loss = 36.48 (0.025 sec)
Epoch 2: loss = 16.91 (0.030 sec)
Epoch 3: loss = 10.26 (0.012 sec)
Epoch 4: loss = 7.19 (0.012 sec)
Epoch 5: loss = 5.47 (0.012 sec)
Epoch 6: loss = 4.39 (0.011 sec)
Epoch 7: loss = 3.66 (0.011 sec)
Epoch 8: loss = 3.12 (0.011 sec)
Epoch 9: loss = 2.72 (0.011 sec)
Epoch 10: loss = 2.41 (0.011 sec)
Epoch 11: loss = 2.16 (0.009 sec)
Epoch 12: loss = 1.96 (0.006 sec)
Epoch 13: loss = 1.79 (0.007 sec)
Epoch 14: loss = 1.65 (0.007 sec)
Epoch 15: loss = 1.53 (0.007 sec)
Epoch 16: loss = 1.42 (0.007 sec)
Epoch 17: loss = 1.33 (0.007 sec)
Epoch 18: loss = 1.25 (0.006 sec)
Epoch 19: loss = 1.18 (0.007 sec)
Epoch 20: loss = 1.11 (0.006 sec)
Epoch 21: loss = 1.05 (0.007 sec)
Epoch 22: loss = 1.00 (0.007 sec)
Epoch 23: loss = 0.95 (0.007 sec)
Epoch 24: loss = 0.91 (0.007 sec)
Epoch 25: loss = 0.87 (0.007 sec)
Epoch 26: loss = 0.84 (0.006 sec)
Epoch 27: loss = 0.80 (0.007 sec)
Epoch 28: loss = 0.77 (0.007 sec)
Epoch 29: loss = 0.74 (0.007 sec)
Epoch 30: loss = 0.72 (0.007 sec)
Epoch 31: loss = 0.69 (0.008 sec)
Epoch 32: loss = 0.67 (0.007 sec)
Epoch 33: loss = 0.65 (0.006 sec)
Epoch 34: loss = 0.63 (0.006 sec)
Epoch 35: loss = 0.61 (0.006 sec)
Epoch 36: loss = 0.59 (0.006 sec)
Epoch 37: loss = 0.57 (0.006 sec)
Epoch 38: loss = 0.56 (0.006 sec)
Epoch 39: loss = 0.54 (0.007 sec)
Epoch 40: loss = 0.53 (0.006 sec)
Epoch 41: loss = 0.52 (0.006 sec)
Epoch 42: loss = 0.50 (0.006 sec)
Epoch 43: loss = 0.49 (0.006 sec)
Epoch 44: loss = 0.48 (0.007 sec)
Epoch 45: loss = 0.47 (0.006 sec)
Epoch 46: loss = 0.46 (0.006 sec)
Epoch 47: loss = 0.45 (0.006 sec)
Epoch 48: loss = 0.44 (0.006 sec)
Epoch 49: loss = 0.43 (0.006 sec)
Basic (non-exhaustive) classifier tests pass
```

# **2 基于神经网络的转移依存句法分析**

这部分与之前的cs224d不同，之前的课程是命名实体识别的例子。这门课程则是用转移依存句法分析作为应用的实例。

依存句法分析一是判断给定的句子是否合乎语法，再是为合乎语法的句子给出句法结构。为了准确做出句子的依存关系，不少学者提出了一些方法，如基于图的方法，基于转换的方法等。这里就是采用基于转换的神经网络方法。

这部分的理论知识不是很熟需要补充。。。
参考资料：

http://www.cipsc.org.cn/qngw/?p=885

https://blog.csdn.net/weixin_38889448/article/details/80009215

http://www.hankcs.com/nlp/parsing/

https://applenob.github.io/gen_dep_parser.html


## **2-1 手工完成Transition-Based Dependency Parsing**

对句子："I parsed this sentence correctly" 以手工方式写出依存句法树，在每个步骤罗列出stack，buffers和transition信息，及其生成的new dependency信息。结果如下：


![2-1.jpg](2-1.jpg)

## **2-2 复杂度**

一个含有n个单词的句子会花费多少个步骤？

解答：

句子中每个单词都必须shift到stack中，再进行一步的reduced，所以2n。即每个单词shift一步，arc一步。


## **2-3 初始化 & parse_step**

PartialParse初始化：
完善`q2_parser_transitions.py`中的`__init __`和`parse_step`函数，以实现`transition mechanics`.


In [ ]:
def __init__(self, sentence):
    """Initializes this partial parse.
    Your code should initialize the following fields:
        self.stack: The current stack represented as a list with the top of the stack as the
                    last element of the list.
        self.buffer: The current buffer represented as a list with the first item on the
                     buffer as the first item of the list
        self.dependencies: The list of dependencies produced so far. Represented as a list of
                tuples where each tuple is of the form (head, dependent).
                Order for this list doesn't matter.
    The root token should be represented with the string "ROOT"
    Args:
        sentence: The sentence to be parsed as a list of words.
                  Your code should not modify the sentence.
    """
    # The sentence being parsed is kept for bookkeeping purposes. Do not use it in your code.
    self.sentence = sentence
    ### YOUR CODE HERE
    self.stack = ['ROOT']
    self.buffer = sentence[:]
    self.dependencies = []
    ### END YOUR CODE

转移动作：

In [ ]:
def parse_step(self, transition):
    """Performs a single parse step by applying the given transition to this partial parse
    Args:
        transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                    and right-arc transitions.
    """
    ### YOUR CODE HERE
    if transition == "S":
        self.stack.append(self.buffer[0])
        self.buffer.pop(0)
    elif transition == "LA":
        self.dependencies.append((self.stack[-1], self.stack[-2]))
        self.stack.pop(-2)
    else:
        self.dependencies.append((self.stack[-2], self.stack[-1]))
        self.stack.pop(-1)
    ### END YOUR CODE

## **2-4 Minibatch Parsing**

得益于TF强大的矩阵加速能力，你可以将多个句子同时传入。算法流程如下：

![2-4.jpg](2-4.jpg)

代码实现如下:

In [ ]:
def minibatch_parse(sentences, model, batch_size):
    """Parses a list of sentences in minibatches using a model.

    Args:
        sentences: A list of sentences to be parsed (each sentence is a list of words)
        model: The model that makes parsing decisions. It is assumed to have a function
               model.predict(partial_parses) that takes in a list of PartialParses as input and
               returns a list of transitions predicted for each parse. That is, after calling
                   transitions = model.predict(partial_parses)
               transitions[i] will be the next transition to apply to partial_parses[i].
        batch_size: The number of PartialParses to include in each minibatch
    Returns:
        dependencies: A list where each element is the dependencies list for a parsed sentence.
                      Ordering should be the same as in sentences (i.e., dependencies[i] should
                      contain the parse for sentences[i]).
    """

    ### YOUR CODE HERE
    # refer: https://github.com/zysalice/cs224/blob/master/assignment2/q2_parser_transitions.py
    partial_parses = [PartialParse(s) for s in sentences]

    unfinished_parse = partial_parses

    while len(unfinished_parse) > 0:
        minibatch = unfinished_parse[0:batch_size]
        # perform transition and single step parser on the minibatch until it is empty
        while len(minibatch) > 0:
            transitions = model.predict(minibatch)
            for index, action in enumerate(transitions):
                minibatch[index].parse_step(action)
            minibatch = [parse for parse in minibatch if len(parse.stack) > 1 or len(parse.buffer) > 0]

        # move to the next batch
        unfinished_parse = unfinished_parse[batch_size:]

    dependencies = []
    for n in range(len(sentences)):
        dependencies.append(partial_parses[n].dependencies)
    ### END YOUR CODE

    return dependencies

## **2-5 Xavier初始化**

为了防止神经元之间过度依赖，常用的技巧之一是Xavier Initialization。给定m×n大小的A，按照[−ϵ,ϵ]区间的均匀分布生成每个元素$A_{ij}$：

$$
\begin{align} 
	\epsilon &= \frac{\sqrt{6}}{\sqrt{m + n}} 
\end{align}
$$
实现如下：

In [ ]:
def xavier_weight_init():
    """Returns function that creates random tensor.
    The specified function will take in a shape (tuple or 1-d array) and
    returns a random tensor of the specified shape drawn from the
    Xavier initialization distribution.
    Hint: You might find tf.random_uniform useful.
    """
    def _xavier_initializer(shape, **kwargs):
        """Defines an initializer for the Xavier distribution.
        Specifically, the output should be sampled uniformly from [-epsilon, epsilon] where
            epsilon = sqrt(6) / <sum of the sizes of shape's dimensions>
        e.g., if shape = (2, 3), epsilon = sqrt(6 / (2 + 3))
        This function will be used as a variable initializer.
        Args:
            shape: Tuple or 1-d array that species the dimensions of the requested tensor.
        Returns:
            out: tf.Tensor of specified shape sampled from the Xavier distribution.
        """
        ### YOUR CODE HERE
        epsilon = np.sqrt(6 / np.sum(shape))
        out = tf.Variable(tf.random_uniform(shape=shape, minval=-epsilon, maxval=epsilon))
        ### END YOUR CODE
        return out
    # Returns defined initializer function.
    return _xavier_initializer

## **2-6 Dropout**

随机地将隐藏层节点的激活值以概率p设为0，然后乘上一个常量 $γ$ ，这个过程可以写作：
$$
\begin{align} 
	\boldsymbol{h}_{drop} &= \lambda \boldsymbol{d} \circ \boldsymbol{h} 
\end{align}
$$

其中，$d∈\{0,1\}^{D_h} (D_h 是隐藏层h 单元数)$ 是一个遮罩向量，每个元素以概率p取0，概率1−p取1。$γ$的选取要保证激活值的期望不变，即对所有$0≤i≤D_h$都有：
$$
\begin{align} 
	\mathbb{E_{p}}[\boldsymbol{h}_{drop}]_{i} &= \boldsymbol{h}_{i} 
\end{align}
$$
那么给定$p，γ$ 到底要取多少呢？
$$\gamma = \frac{1}{p}$$



## ** 2-7 Adam Optimizer**

标准的SGD更新规则：
$$
\begin{align} 
	\boldsymbol{\theta} \leftarrow \boldsymbol{\theta} - \alpha \nabla_{\boldsymbol{\theta}} J_{minibatch}(\boldsymbol{\theta}) 
\end{align}
$$
Adam用的是更加复杂的更新规则，有两个额外的步骤：

1、动量更新，记录一个历史梯度的均值（速度）$m$：
$$
\begin{align} 
		\boldsymbol{m} &\leftarrow \beta_{1}\boldsymbol{m} + (1 - \beta_{1}) \nabla_{\boldsymbol{\theta}} J_{minibatch}(\boldsymbol{\theta})\\ 
		% 
		\boldsymbol{\theta} &\leftarrow \boldsymbol{\theta} - \alpha\boldsymbol{m} 
	\end{align}
$$
其中$β_1$ 是一个 0 到 1 (经常取 0.9)的超参数。由于$β_1$很接近1，所以每次更新量与前一次大致相同，动量减小了更新量的方差，避免了梯度的剧烈震荡。

2、Adam中的学习率还是自适应的，通过记录历史梯度的长度的平方v来得到:
$$
\begin{align} 
			\boldsymbol{m} &\leftarrow \beta_{1}\boldsymbol{m} + (1 - \beta_{1}) \nabla_{\boldsymbol{\theta}} J_{minibatch}(\boldsymbol{\theta})\\ 
			% 
			\boldsymbol{v} &\leftarrow \beta_{2}\boldsymbol{v} + (1 - \beta_{2})( \nabla_{\boldsymbol{\theta}} J_{minibatch}(\boldsymbol{\theta}) \circ \nabla_{\boldsymbol{\theta}} J_{minibatch}(\boldsymbol{\theta})) \\ 
			% 
			\boldsymbol{\theta} &\leftarrow \boldsymbol{\theta} - \alpha\boldsymbol{m} / \sqrt{\boldsymbol{v}} 
	\end{align}
$$
其中$∘$ 和 $/$ 都是 elementwise运算。
在Adam中，那些梯度平均较小的参数的更新量会更大。也就是说在损失函数相对于它们的梯度很小的地方也能快速移动，能够快速逃离高原。

## **2-8 实现Neural Dependency Parser**

要求在开发集上的UAS大于88，loss小于0.07。

就贴个最主要的方法吧：

In [ ]:
def add_prediction_op(self):
    """Adds the 1-hidden-layer NN:
        h = Relu(xW + b1)
        h_drop = Dropout(h, dropout_rate)
        pred = h_dropU + b2
    Note that we are not applying a softmax to pred. The softmax will instead be done in
    the add_loss_op function, which improves efficiency because we can use
    tf.nn.softmax_cross_entropy_with_logits
    Use the initializer from q2_initialization.py to initialize W and U (you can initialize b1
    and b2 with zeros)
    Hint: Here are the dimensions of the various variables you will need to create
                W:  (n_features*embed_size, hidden_size)
                b1: (hidden_size,)
                U:  (hidden_size, n_classes)
                b2: (n_classes)
    Hint: Note that tf.nn.dropout takes the keep probability (1 - p_drop) as an argument. 
        The keep probability should be set to the value of self.dropout_placeholder
    Returns:
        pred: tf.Tensor of shape (batch_size, n_classes)
    """
    x = self.add_embedding()
    ### YOUR CODE HERE
    xavier = xavier_weight_init()
    with tf.variable_scope("transformation"):
        b1 = tf.Variable(tf.random_uniform([self.config.hidden_size,]))
        b2 = tf.Variable(tf.random_uniform([self.config.n_classes]))
        W = xavier([self.config.n_features * self.config.embed_size, self.config.hidden_size])
        U = xavier([self.config.hidden_size, self.config.n_classes])
        z1 = tf.matmul(x,W) + b1
        h = tf.nn.relu(z1)
        h_drop = tf.nn.dropout(h,self.dropout_placeholder)
        pred = tf.matmul(h_drop, U) + b2
    ### END YOUR CODE
    return pred

经典的三层网络吧，加了个dropout。在完整数据集上跑完10个epoch花了1032s，并没有题目上说的要一个小时。
输出：

```
Epoch 10 out of 10
924/924 [============================>.] - ETA: 0s - train loss: 0.0664Evaluating on dev set
- dev UAS: 88.45
New best dev UAS! Saving model in ./data/weights/parser.weights

================================================================================
TESTING
================================================================================
Restoring the best model weights found on the dev set
Final evaluation on test set
- test UAS: 89.16
Writing predictions
Done!
cost= 1032.8432188034058

```

可以自己再加个L2正则或者信手调调参，看能否继续提高。

开发集loss升去了（损失函数多了一项）。考虑到Dropout其实已经是一种正则化，所以这里L2正则化其实没有什么用。
如同[《基于神经网络的高性能依存句法分析器》](http://www.hankcs.com/nlp/parsing/neural-network-based-dependency-parser.html)所述，原论文使用了很多技巧，比如将词性等标签也向量化了，分值当然更高。另外，这里使用的词向量仅有50维，只训练了10个epoch，自然没有论文高。


# 3 RNN和语言模型

给定一个词语的序列：

$$
\boldsymbol{x}^{(1)}, \boldsymbol{x}^{(2)}, \dots, \boldsymbol{x}^{(t)}
$$

语言模型的任务就是通过建模

$$
\begin{align} 
	P(\boldsymbol{x}^{(t+1)} &= \boldsymbol{v}_{j} \vert \boldsymbol{x}^{(t)}, \dots, \boldsymbol{x}^{(1)}) 
\end{align}
$$

来预测下一个单词$\boldsymbol{x}^{(t+1)}$，其中$\boldsymbol{v}_{j}$是词表中的一个单词。

对RNN来讲，有：

$$
\begin{align} 
	\boldsymbol{e}^{(t)} &= \boldsymbol{x}^{(t)}\boldsymbol{L} \\ 
	% 
	\boldsymbol{h}^{(t)} &= \text{sigmoid} \left( \boldsymbol{h}^{(t-1)}\boldsymbol{H} + \boldsymbol{e}^{(t)}\boldsymbol{I} + \boldsymbol{b}_{1} \right)\\ 
	% 
	\hat{\boldsymbol{y}}^{(t)} &= \text{softmax}  \left( \boldsymbol{h}^{(t)}\boldsymbol{U} + \boldsymbol{b}_{2} \right) \\ 
	% 
	\bar{P}\left( \boldsymbol{x}^{(t+1)} = \boldsymbol{v}_{j} \vert \boldsymbol{x}^{(t)}, \dots, \boldsymbol{x}^{(1)} \right) &= \hat{\boldsymbol{y}}_{j}^{(t)} 
\end{align}
$$

其中$\boldsymbol{h}^{(0)} = \boldsymbol{h}_{0} \in \mathbb{R}^{D_{k}}$是隐藏层的初始状态，$\boldsymbol{L}$是lookup-table，即词嵌入矩阵。
$I$ 是输入词表征矩阵，$H$是隐藏转化矩阵，还有$U$是输出词表征矩阵。$b1,b2$是偏置值。

参数维度如下：

$$
\begin{align} 
	&\boldsymbol{L} \in \mathbb{R}^{\vert V \vert \times d} 
	% 
	&\boldsymbol{H} \in \mathbb{R}^{D_{h} \times D_{h}} \\ 
	% 
	% 
	&\boldsymbol{I} \in \mathbb{R}^{d \times D_{h}} 
	% 
	&\boldsymbol{b}_{1} \in \mathbb{R}^{D_{h}} \\ 
	% 
	% 
	& \boldsymbol{U} \in \mathbb{R}^{D_{h} \times \vert V \vert} 
	% 
	& \boldsymbol{b}_{2} \in \mathbb{R}^{\vert V \vert} 
\end{align}
$$

d是嵌入维度，$|V|$是词表大小，$D_{h}$是隐藏单元数，即隐层的维数。

输出向量$\hat{\boldsymbol{y}}^{(t)} \in \mathbb{R}^{\vert V \vert}$是词表上的概率分布。模型通过最小化交叉熵损失训练：

$$
\begin{align} 
	J^{(t)}(\theta) = CE(\boldsymbol{y}^{(t)} , \hat{\boldsymbol{y}}^{(t)}) &= - \sum_{j=1}^{\vert V \vert} y_{j}^{(t)} log(\hat{y}_{j}^{(t)}) 
\end{align}
$$

其中，$y^{(t)}$是与目标词（这里表示为$x_{t+1}$）对应的one-hot向量。正如注释[2]所描述的，它是一个对于单个样本点的损失率，然后我们对序列中全部样例的交叉熵损失值做求和（或求平均）计算，对数据集中的全部序列[4]均采取上述操作以衡量模型的性能。

## 3-1 困惑度和交叉熵损失

按惯例，语言模型的效果用困惑度衡量，其定义如下：

$$
{PP}^{(t)}\left(y^{(t)},\hat{y}^{(t)}\right)=\frac{1}{\bar{P}(x_{t+1}^\mathrm{pred}=x_{t+1}|x_t,\ldots,x_1)}=\frac{1}{\sideset{}{_{j=1}^{|V|}} \sum{y_j^{(t)}}\cdot\hat{y}^{(t)}}\tag{16}
$$

即通过模型分布$\bar{P}$得到词预测正确的逆概率。给出你是如何从交叉熵的损失率中得到困惑度的（提示：不要忘了向量$y^{(t)}$是one-hot形式的！），因此如果要进行最小化操作（从算术上）意味着（从几何上讲）交叉熵损失率将被最小化，就得到了在训练集上的困惑度。在这一部分要处理的是一个非常短小的问题-所以并不需要过大的困惑度！ 

对于一个词库$|V|$中的词汇，如果你的模型做出的预测是完全随机的，你将如何预测你的困惑度呢？计算词库规模为$|V|=2000和|V|=10000$时对应的交叉熵损失率，并且将其作为基准牢记于心。


解答：

使用的样例$y^{(t)}$为one-hot模型，且假定$y_i^{(t)}$是$y^{(t)}$中唯一的非零元素。于是，记：

$$
CE(y^{(t)},\hat{y}^{(t)})=-\log\hat{y}_i^{(t)}=\log\frac{1}{\hat{y}^{(t)}}
$$

$$
PP^{(t)}(y^{(t)},\hat{y}^{(t)})=\frac{1}{\hat{y}_i^{(t)}}
$$

因此，上式可以被联立为如下形状：

$$
CE(y^{(t)},\hat{y}^{(t)})=\log PP^{(t)}(y^{(t)},\hat{y}^{(t)})
$$

这条公式展示了交叉熵在数学意义上的最小化目标，同时表达了困惑度在几何意义上的最小值。当模型的预测值完全随机化时，会有
$E[\hat{y}_i^{(t)}]=\frac{1}{|V|}$。所给的向量$y^{(t)}$是one-hot模型，所对应困惑度的期望值是$|V|$。由于困惑度的对数即为交叉熵，那么交叉熵的期望值在面对上述两个词库规模时分别应为$log2000≈7.6l和log10000≈9.21。$

## 3-2 推导梯度

在时刻t关于单点全部模型参数的梯度计算如下：
$$
\begin{align} 
	\frac{ \partial J^{(t)} }{ \partial \boldsymbol{b}_{2} } 
\quad 
	\frac{ \partial J^{(t)} }{ \partial \boldsymbol{L}_{x^{(t)}} } 
\quad 
	\frac{ \partial J^{(t)} }{ \partial \boldsymbol{I} } \bigg\rvert_{(t)} 
\quad 
	\frac{ \partial J^{(t)} }{ \partial \boldsymbol{H} } \bigg\rvert_{(t)} 
\quad 
\frac{\partial J^{(t)}}{\partial \boldsymbol{h}^{(t-1)}} 
\end{align}
$$

其中，$\boldsymbol{L}_{x^{(t)}}$是词嵌入矩阵$\boldsymbol{L}$中对应到当前处理词汇$x^{(t)}$的列，符号$|(t)$表示时刻t该参数的显式梯度。（同样地，$\boldsymbol{h}^{(t-1)}$的取值是固定的，而且你现在也不需要在早先的迭代时刻中实现反向传播算法——这是第3小节的任务）。

此外，还要计算代表前层隐层权值的导数：

$$
\frac{\partial{J^{(t)}}}{\partial{\boldsymbol{h}^{(t-1)}}}
$$

![3-2.jpg](3-2.jpg)

**推导过程：**

为方便，记：

$$
\boldsymbol{f}_{1}^{(t)} = \boldsymbol{h}^{(t-1)}\boldsymbol{H} + e^{(t)}\boldsymbol{I} + \boldsymbol{b}_{1},
\boldsymbol{f}_{2}^{(t)} = \boldsymbol{h}^{(t)}\boldsymbol{U} + \boldsymbol{b}_{2}
$$



调用函数$\frac{d}{dz}\mathrm{sigmoid}(z)=\mathrm{sigmoid}(z)(1-\mathrm{sigmoid}(z))$

计算误差：

$$
\begin{align} 
	\boldsymbol{\delta}_{2}^{(t)} &= \frac{\partial J^{(t)}}{\partial \boldsymbol{f}_{2}^{(t)}} % 
					  	   = \hat{\boldsymbol{y}}^{(t)} - \boldsymbol{y}^{(t)} \nonumber \\ 
	% 
	% 
	\boldsymbol{\delta}_{1}^{(t)} 
	&= \frac{\partial J^{(t)}}{\partial \boldsymbol{f}_{1}^{(t)}} \\ 
	&=\frac{\partial \boldsymbol{J}^{(t)}}{\partial \boldsymbol{f}_2^{(t)}} \cdot \frac{\partial \boldsymbol{f}_{2}^{(t)}}{\partial \boldsymbol{h}^{(t)}} \cdot \frac{\partial \boldsymbol{h}^{(t)}}{\partial \boldsymbol{f}_{1}^{(t)}} \\ 
					  	   &= \boldsymbol{\delta}_{2}^{(t)} \cdot \frac{\partial \boldsymbol{f}_{2}^{(t)}}{\partial \boldsymbol{h}^{(t)}} \cdot			\frac{\partial \boldsymbol{h}^{(t)}}{\partial \boldsymbol{f}_{1}^{(t)}} \nonumber \\ 
					  	  &= \boldsymbol{\delta}_{2}^{(t)} \cdot \boldsymbol{U}^{T} \circ \boldsymbol{h}^{(t)} \circ (1 - \boldsymbol{h}^{(t)}) \nonumber 
\end{align}
$$


基于上述的结果，可以得出：

$$
\begin{align} 
	\frac{\partial J^{(t)}}{\partial \boldsymbol{b}_{2}} &= % 
		\frac{\partial J^{(t)}}{\partial \boldsymbol{f}_{2}^{(t)}} \frac{\partial \boldsymbol{f}_{2}^{(t)}}{\partial \boldsymbol{b}_{2}} \nonumber \\ 
	% 
	% 
										   &= \boldsymbol{\delta}_{2}^{(t)} = \hat{\boldsymbol{y}}^{(t)} - \boldsymbol{y}^{(t)} \nonumber 
\end{align}
$$

$$
\begin{align} 
	\frac{\partial J^{(t)}}{\partial \boldsymbol{L}_{x^{(t)}}} &= % 
		\frac{\partial J^{(t)}}{\partial \boldsymbol{f}_{1}^{(t)}} \frac{\partial \boldsymbol{f}_{1}^{(t)}}{\partial \boldsymbol{e}^{(t)}} % 
			\frac{\partial \boldsymbol{e}^{(t)}}{\partial \boldsymbol{L}_{x^{(t)}}} \nonumber \\ 
	% 
	% 
												 &= \boldsymbol{\delta}_{1}^{(t)} \boldsymbol{I}^{T} \nonumber 
\end{align}
$$

这里利用到了one-hot向量时$\boldsymbol{e}^{(t)}= \boldsymbol{L}_{x^{(t)}}$

$$
\begin{align} 
	\frac{\partial J^{(t)}}{\partial \boldsymbol{I}} &= % 
		\frac{\partial J^{(t)}}{\partial \boldsymbol{f}_{1}^{(t)}} \frac{\partial \boldsymbol{f}_{1}^{(t)}}{\partial \boldsymbol{I}} \nonumber \\ 
	% 
	% 
									   &=  (\boldsymbol{e}^{(t)})^{T} \boldsymbol{\delta}_{1}^{(t)}  \nonumber 
\end{align}
$$

$$
\begin{align} 
	\frac{\partial J^{(t)}}{\partial \boldsymbol{H}} &= % 
		\frac{\partial J^{(t)}}{\partial \boldsymbol{f}_{1}^{(t)}} \frac{\partial \boldsymbol{f}_{1}^{(t)}}{\partial \boldsymbol{H}} \nonumber \\ 
		% 
		% 
									   &= (\boldsymbol{h}^{(t-1)})^{T} \boldsymbol{\delta}_{1}^{(t)} \nonumber 
\end{align}
$$

$$
\begin{align} 
	\frac{\partial J}{\partial \boldsymbol{h}^{(t-1)}} &= % 
		\frac{\partial J^{(t)}}{\partial \boldsymbol{f}_{1}^{(t)}} \frac{\partial \boldsymbol{f}_{1}^{(t)}}{\partial \boldsymbol{h}^{(t-1)}} \nonumber \\ 
				% 
				% 
											   &=  \boldsymbol{\delta}_{1}^{(t)} \boldsymbol{H}^{T} \nonumber 
\end{align}
$$

## 3-3 反向传播

绘制下面三次迭代所“展开”的网络，并且计算迭代时后向传播的梯度：

$$
\begin{align} 
	\frac{ \partial J^{(t)} }{ \partial \boldsymbol{L}_{x^{(t-1)}} } 
	\quad 
	\frac{ \partial J^{(t)} }{ \partial \boldsymbol{I} } \bigg\rvert_{(t-1)} 
	\quad 
	\frac{ \partial J^{(t)} }{ \partial \boldsymbol{H} } \bigg\rvert_{(t-1)} 
\end{align}
$$

利用
$$
\boldsymbol{\delta}^{(t-1)} = \frac{\partial J^{(t)}}{\partial \boldsymbol{h}^{(t-1)}}
$$
将这些梯度表达成残差的形式,来简化结果。

真实梯度需要运用反向传播到$t=0$，而实践中经常只运行固定的$\tau \approx 5-10$步。

**网络unroll的结果如下：**

![3-3.jpg](3-3.jpg)

**推导过程：**

引入如下记号：
$$
\sigma'(\boldsymbol{f}_{1}^{(t-1)}) = \frac{\partial \boldsymbol{h}^{(t-1)}}{\partial \boldsymbol{f}_{1}^{(t-1)}} = diag\left(\boldsymbol{h}^{(t-1)} \circ (1 - \boldsymbol{h}^{(t-1)}) \right)
$$


对于$\frac{\partial J ^{(t)}}{\partial \boldsymbol{L}_{x^{(t-1)}}}$有：

$$
\begin{align} 
	\frac{\partial J ^{(t)}}{\partial \boldsymbol{L}_{x^{(t-1)}}} &= % 
		\frac{\partial J ^{(t)}}{\partial \boldsymbol{h}^{(t-1)}} % 
		\frac{\partial \boldsymbol{h}^{(t-1)}}{\partial \boldsymbol{f}_{1}^{(t-1)}} % 
		\frac{\partial \boldsymbol{f}_{1}^{(t-1)}}{\partial \boldsymbol{L}_{x^{(t-1)}}} \nonumber \\ 
	% 
		&= \boldsymbol{\delta}^{(t-1)} \sigma'(\boldsymbol{f}_{1}^{(t-1)}) \boldsymbol{I}^{T} \nonumber 
\end{align}
$$


对于$\boldsymbol{I}$有：

$$
\begin{align} 
	\frac{\partial J ^{(t)}}{\partial \boldsymbol{I}} &= % 
		\frac{\partial J ^{(t)}}{\partial \boldsymbol{h}^{(t-1)}} % 
		\frac{\partial \boldsymbol{h}^{(t-1)}}{\partial \boldsymbol{f_1}^{(t-1)}} % 
		\frac{\partial \boldsymbol{f_1}^{(t-1)}}{\partial \boldsymbol{I}} \nonumber \\ 
	% 
		&=  (\boldsymbol{e}^{(t-1)})^{T} \boldsymbol{\delta}_{1}^{(t-1)} \sigma'(\boldsymbol{f_1}^{(t-1)}) \nonumber 
\end{align}
$$

对于$\boldsymbol{H}$有：

$$
\begin{align} 
	\frac{\partial J ^{(t)}}{\partial \boldsymbol{H}} &= % 
		\frac{\partial J ^{(t)}}{\partial \boldsymbol{h}^{(t-1)}} % 
		\frac{\partial \boldsymbol{h}^{(t-1)}}{\partial \boldsymbol{f_1}^{(t-1)}} % 
		\frac{\partial \boldsymbol{f_1}^{(t-1)}}{\partial \boldsymbol{H}} \nonumber \\ 
	% 
		&= (\boldsymbol{h}^{(t-2)})^{T} \boldsymbol{\delta}^{(t-1)} \sigma'(\boldsymbol{f_1}^{(t-1)}) \nonumber 
\end{align}
$$

## 3-4 计算复杂度

对于给定的$h^{(t-1)}$,执行一轮前向传播计算$J^{(t)}(\theta)$需要多少操作？执行一轮后向传播又会如何呢？执行$\tau$轮呢？对于维度参数组$d,D_h$和$|V|$，分别来自输入词向量到隐藏层的权值矩阵(即稠密向量，词向量的维度)、隐藏层到隐藏层的权值矩阵、隐藏层到输出的权值矩阵(即词典的长度)。
使用符号“大写-O”来表示你的答案。

解答：

前向传播：
$$
\begin{align} 
	 O\left( (d \times D_{h}) + D_{h}^{2} + (D_{h} \times |V|) \right) \nonumber 
\end{align}
$$

反向传播：

如果对所有单词计算梯度：

$$
\begin{align} O\left( \tau \left((d \times D_{h}) + D_{h}^{2} +  D_{h} \times |V| \right)\right) \nonumber 
\end{align}
$$

这是因为反向传播$\tau$次的路径上，残差来自$\tau$个输出单词。

如果对单个单词计算梯度：

$$
\begin{align} O\left( \tau \left((d \times D_{h}) + D_{h}^{2}  \right) +  D_{h} \times |V| \right) \nonumber \end{align}
$$
此时反向传播$\tau$次的路径上，来自$\tau$个输出单词的残差中对该单词之外的导数为0。

由于词表远远大于隐藏层节点数$|V|>>D_h$，所以softmax层的$O(D_{h} \times |V|)$是瓶颈。一个解决办法是用NCE代替昂贵的交叉熵损失函数。

这部分居然没有代码要求，看了下之前的课程是有的。感兴趣的童鞋可以参考`cs224d_assignment2_solution`文件中的`q3_RNNLM.py`进行学习。

在代码`q3_RNNLM.py`中实现的模型。其中已经实现了数据加载器和其它的初始化功能代码执行命令行`python q3_RNNLM.py`将运行该模型。注意，在运行过程中你不能用到tensorflow库的内置函数，如`rnn_cell`模块。

在`ptb－train`数据集中训练该模型，其中包含`Penn Treebank`中WSJ语料集的前20节语料。正如在第2部分中所说的，你应该最大限度地提高该模型在实际生产中数据集上的泛化性能（即最小化交叉熵损失率）。关于模型的性能，我们将通过未知但类似的句子集来进行评估。

作业1和该作业中的q2部分所示的神经网络是判别模型：他们接收数据之后，对其做出预测。前面你实现的RNNLM模型是一个生成式模型，因为它建模了数据序列$x1,\dots,xn$的分布状况。这就意味着我们不仅能用它去评估句子生成概率，而且还能通过它生成概率最优的语句！

训练完成后，在代码`q3 RNNLM.py` 中实现函数`generate_text()`。该函数首先运行RNN前向算法，在起始标记`<eos>`处建立索引，然后从每轮迭代的$\hat{y}^{(t)}$分布对新词$x_{t+1}$进行采样。然后把该词作为下一步的输入，重复该操作直至该模型遇到一个结束标志（结束标志记为`</eos>`）。
